-------------------------------------------------------------------------------

CHIBO 2024

An attempt to static calculate the forces in (x,y,z) at all the suspension pickup points for various roll/jounce effects

18.12.2024: Created

03.01.2025: Outboard points calc working, moved functions to susprog.py and added loop 

-------------------------------------------------------------------------------

In [13]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

import plotly.graph_objects as go
import matplotlib.pyplot as plt

import susprog                      # Python file containing the equations for the loop 

In [14]:
# Declare motions for evaluation

# Define maximum roll and jounce inputs
body_roll = -10  # Maximum roll angle in degrees
body_jounce = 0.05  # Maximum jounce in meters (5 cm)

# Generate roll and jounce motion inputs
roll_angles = np.linspace(0, np.radians(body_roll), 100)  # 0 to max roll angle in 100 steps
jounce_displacements = np.linspace(0, body_jounce, 100)  # 0 to max jounce in 100 steps

In [15]:
# Suspension pickup points
#        x      y    z
upper_a_arm = np.array([
    [0.0015, 0.5065, 0.315],  # Outer point (converted from mm to m)
    [-0.135, 0.240, 0.220],   # Inner leading point
    [0.180, 0.255, 0.255]     # Inner trailing point
])

lower_a_arm = np.array([
    [-0.0175, 0.519, 0.137],  # Outer point
    [-0.180, 0.165, 0.087],   # Inner leading point
    [0.180, 0.170, 0.087]     # Inner trailing point
])

tie_rod = np.array([
    [0.060, 0.515, 0.160],  # Outer point
    [0.070, 0.195, 0.100]   # Inner point
])

tire_cop = np.array([-0.005, 0.525, 0])  # Converted from mm to m
force_vector = np.array([50, 500, -2000.0])  # Force at the CoP (N)


In [16]:
# Calculate lengths for upper and lower A-arms and tie rod
upper_lengths = [
    np.linalg.norm(upper_a_arm[1] - upper_a_arm[0]),  # Upper outer to inner leading
    np.linalg.norm(upper_a_arm[2] - upper_a_arm[0]),  # Upper outer to inner trailing
    np.linalg.norm(upper_a_arm[0] - lower_a_arm[0])   # Upper outer to lower outer
]

lower_lengths = [
    np.linalg.norm(lower_a_arm[1] - lower_a_arm[0]),  # Lower outer to inner leading
    np.linalg.norm(lower_a_arm[2] - lower_a_arm[0]),  # Lower outer to inner trailing
    np.linalg.norm(lower_a_arm[0] - tire_cop)         # Lower outer to tire center of pressure
]

tie_length = [
    np.linalg.norm(tie_rod[1] - tie_rod[0]),          # Tie rod outer to inner leading
    np.linalg.norm(upper_a_arm[0] - tie_rod[0]),      # Tie rod outer to upper outer
    np.linalg.norm(lower_a_arm[0] - tie_rod[0])       # Tie rod outer to lower outer
]

In [17]:
# Roll centre calc @ static
rc, geo_roll_centre = susprog.calculate_roll_center(upper_a_arm, lower_a_arm, tire_cop)

# Print outputs
# print("Roll Center [x, y, z]:", rc)
# print("\nGeometry Data:")
# for key, value in geo_roll_centre.items():
#     print(f"{key}:", value)

In [18]:
# Do movement calculation

# Step 1: Calculate roll centre @ Static
rc, geo_roll_centre = susprog.calculate_roll_center(upper_a_arm, lower_a_arm, tire_cop)
roll_axis = np.array([1, 0, 0])         # Example roll axis (x-axis)

# Step 2: Initialize rotated arms as a copy of the original
rotated_upper = upper_a_arm.copy()
rotated_lower = lower_a_arm.copy()
rotated_tie = tie_rod.copy()

# Rotate the inner points
rotated_upper_inner = susprog.rotate_points(upper_a_arm[1:], rc, roll_axis, roll_angles[-1])
rotated_lower_inner = susprog.rotate_points(lower_a_arm[1:], rc, roll_axis, roll_angles[-1])
rotated_tie_inner = susprog.rotate_points(tie_rod[1:], rc, roll_axis, roll_angles[-1])

rotated_upper[1:] = rotated_upper_inner
rotated_lower[1:] = rotated_lower_inner
rotated_tie[1:] = rotated_tie_inner

del rotated_upper_inner
del rotated_lower_inner
del rotated_tie_inner

# Lower A-arm
points_temp = np.vstack((rotated_lower,tire_cop))
rotated_lower[0:1] = susprog.solve_outer_position(points_temp, lower_lengths)
del points_temp

# Upper A-arm
points_temp = np.vstack((rotated_upper,rotated_lower[0:]))
rotated_upper[0:1] = susprog.solve_outer_position(points_temp, upper_lengths)
del points_temp

# Tie rod
points_temp = np.vstack((rotated_tie, rotated_upper[0:1], rotated_lower[0:1]))
rotated_tie[0:1] = susprog.solve_outer_position(points_temp, tie_length)
del points_temp



In [19]:
def plot_suspension_interactive(upper, lower, upper_rot, low_rot, tie, tie_rot, cop):
    fig = go.Figure()

    # Upper A-arm
    fig.add_trace(go.Scatter3d(
        x=[upper[0, 0], upper[1, 0]], y=[upper[0, 1], upper[1, 1]], z=[upper[0, 2], upper[1, 2]],
        mode='lines+markers',
        name='Upper A-arm (Outer to Inner Leading)',
        marker=dict(size=6, color='blue'),
        line=dict(color='blue', width=2)
    ))
    fig.add_trace(go.Scatter3d(
        x=[upper[0, 0], upper[2, 0]], y=[upper[0, 1], upper[2, 1]], z=[upper[0, 2], upper[2, 2]],
        mode='lines+markers',
        name='Upper A-arm (Outer to Inner Trailing)',
        line=dict(color='blue', width=2)
    ))
    fig.add_trace(go.Scatter3d(
        x=[upper[1, 0], upper[2, 0]], y=[upper[1, 1], upper[2, 1]], z=[upper[1, 2], upper[2, 2]],
        mode='lines',
        name='Upper A-arm (Inner leading to Inner Trailing)',
        line=dict(color='blue', width=2, dash='dot')
    ))
    # Rotated Upper A-arm
    fig.add_trace(go.Scatter3d(
        x=[upper_rot[0, 0], upper_rot[1, 0]], y=[upper_rot[0, 1], upper_rot[1, 1]], z=[upper_rot[0, 2], upper_rot[1, 2]],
        mode='lines+markers',
        name='Upper A-arm (Rot Outer to Inner Leading)',
        marker=dict(size=6, color='purple'),
        line=dict(color='purple', width=2)
    ))
    fig.add_trace(go.Scatter3d(
        x=[upper_rot[0, 0], upper_rot[2, 0]], y=[upper_rot[0, 1], upper_rot[2, 1]], z=[upper_rot[0, 2], upper_rot[2, 2]],
        mode='lines+markers',
        name='Upper A-arm (Rot Outer to Inner Trailing)',
        marker=dict(size=6, color='purple'),
        line=dict(color='purple', width=2)
    ))
    
    fig.add_trace(go.Scatter3d(
        x=[upper_rot[1, 0], upper_rot[2, 0]], y=[upper_rot[1, 1], upper_rot[2, 1]], z=[upper_rot[1, 2], upper_rot[2, 2]],
        mode='lines',
        name='Upper A-arm (Rot Inner leading to Inner Trailing)',
        line=dict(color='purple', width=2, dash='dot')
    ))
    
    # Lower A-arm
    fig.add_trace(go.Scatter3d(
        x=[lower[0, 0], lower[1, 0]], y=[lower[0, 1], lower[1, 1]], z=[lower[0, 2], lower[1, 2]],
        mode='lines+markers',
        name='Lower A-arm (Outer to Inner Leading)',
        marker=dict(size=6, color='green'),
        line=dict(color='green', width=2)
    ))
    fig.add_trace(go.Scatter3d(
        x=[lower[0, 0], lower[2, 0]], y=[lower[0, 1], lower[2, 1]], z=[lower[0, 2], lower[2, 2]],
        mode='lines+markers',
        name='Lower A-arm (Outer to Inner Trailing)',
        line=dict(color='green', width=2)
    ))
    fig.add_trace(go.Scatter3d(
        x=[lower[1, 0], lower[2, 0]], y=[lower[1, 1], lower[2, 1]], z=[lower[1, 2], lower[2, 2]],
        mode='lines',
        name='lower A-arm (Inner leading to Inner Trailing)',
        line=dict(color='green', width=2, dash='dot')
    ))
    
    # Rotated Lower A-arm
    fig.add_trace(go.Scatter3d(
        x=[low_rot[0, 0], low_rot[1, 0]], y=[low_rot[0, 1], low_rot[1, 1]], z=[low_rot[0, 2], low_rot[1, 2]],
        mode='lines+markers',
        name='lower rotated A-arm (Outer to Inner Leading)',
        marker=dict(size=6, color='cyan'),
        line=dict(color='cyan', width=2)
    ))
    fig.add_trace(go.Scatter3d(
        x=[low_rot[0, 0], low_rot[2, 0]], y=[low_rot[0, 1], low_rot[2, 1]], z=[low_rot[0, 2], low_rot[2, 2]],
        mode='lines+markers',
        name='lower rotated A-arm (Outer to Inner Trailing)',
        line=dict(color='cyan', width=2)
    ))
    fig.add_trace(go.Scatter3d(
        x=[low_rot[1, 0], low_rot[2, 0]], y=[low_rot[1, 1], low_rot[2, 1]], z=[low_rot[1, 2], low_rot[2, 2]],
        mode='lines',
        name='lower rotated A-arm (Inner leading to Inner Trailing)',
        line=dict(color='cyan', width=2, dash='dot')
    ))

    # Tie rod
    fig.add_trace(go.Scatter3d(
        x=[tie[0, 0], tie[1, 0]], y=[tie[0, 1], tie[1, 1]], z=[tie[0, 2], tie[1, 2]],
        mode='lines+markers',
        name='Tie Rod',
        marker=dict(size=6, color='red'),
        line=dict(color='red', width=2)
    ))

    # Rotated Tie rod
    fig.add_trace(go.Scatter3d(
        x=[tie_rot[0, 0], tie_rot[1, 0]], y=[tie_rot[0, 1], tie_rot[1, 1]], z=[tie_rot[0, 2], tie_rot[1, 2]],
        mode='lines+markers',
        name='Rotated Tie Rod',
        marker=dict(size=6, color='orange'),
        line=dict(color='orange', width=2)
    ))
    # Tire CoP
    fig.add_trace(go.Scatter3d(
        x=[cop[0]], y=[cop[1]], z=[cop[2]],
        mode='markers',
        name='Tire CoP',
        marker=dict(size=8, color='orange')
    ))

    # Set axis labels
    fig.update_layout(
        title="Interactive Suspension Geometry (Corrected Connections)",
        scene=dict(
            xaxis_title="X (m)",
            yaxis_title="Y (m)",
            zaxis_title="Z (m)"
        )
    )

    fig.show()

# Example usage
plot_suspension_interactive(upper_a_arm, lower_a_arm, rotated_upper, rotated_lower, tie_rod, rotated_tie, tire_cop)


In [23]:
# Initialize results matrix

# Columns: [roll_angle (rad), jounce (m), suspension_positions (1x9), rc_x, rc_y, rc_z]
results = np.zeros((len(roll_angles), 23))  # 100 rows, 23 columns
rc, geo_roll_centre = susprog.calculate_roll_center(upper_a_arm, lower_a_arm, tire_cop)

# Roll axis (example: x-axis)
roll_axis = np.array([1, 0, 0])

# Initialize rotated arms as a copy of the original
rotated_upper = upper_a_arm.copy()
rotated_lower = lower_a_arm.copy()
rotated_tie = tie_rod.copy()

# Loop through each roll and jounce input
for i, (roll_angle, jounce_disp) in enumerate(zip(roll_angles, jounce_displacements)):
    # Step 1: Rotate the inner points of the arms and tie rod
    rotated_upper_inner = susprog.rotate_points(upper_a_arm[1:], rc, roll_axis, roll_angle)
    rotated_lower_inner = susprog.rotate_points(lower_a_arm[1:], rc, roll_axis, roll_angle)
    rotated_tie_inner = susprog.rotate_points(tie_rod[1:], rc, roll_axis, roll_angle)

    # Step 2: Apply the updated inner points to the arms
    rotated_upper[1:] = rotated_upper_inner
    rotated_lower[1:] = rotated_lower_inner
    rotated_tie[1:] = rotated_tie_inner

    # Step 3: Update outer points of lower, upper, and tie rod
    # Lower A-arm
    points_temp = np.vstack((rotated_lower, tire_cop))
    rotated_lower[0:1] = susprog.solve_outer_position(points_temp, lower_lengths)
    del points_temp

    # Upper A-arm
    points_temp = np.vstack((rotated_upper, rotated_lower[0:]))
    rotated_upper[0:1] = susprog.solve_outer_position(points_temp, upper_lengths)
    del points_temp

    # Tie rod
    points_temp = np.vstack((rotated_tie, rotated_upper[0:1], rotated_lower[0:1]))
    rotated_tie[0:1] = susprog.solve_outer_position(points_temp, tie_length)
    del points_temp

    # Step 4: Calculate the new roll center
    rc, geo_roll_centre = susprog.calculate_roll_center(rotated_upper, rotated_lower, tire_cop)

    # Step 5: Store the results
    results[i, 0] = roll_angle  # Roll angle (in radians)
    results[i, 1] = jounce_disp  # Jounce displacement
    results[i, 2:11] = rotated_lower.flatten()  # Lower A-arm positions (3x3 -> 1x9)
    results[i, 11:20] = rotated_upper.flatten()  # Upper A-arm positions (3x3 -> 1x9)
    results[i, 20:23] = rc  # Roll center (1x3)

# Convert results to a DataFrame for easier visualization and saving
columns = [
    "roll_angle", "jounce", 
    "lower_outer_x", "lower_outer_y", "lower_outer_z", 
    "lower_inner1_x", "lower_inner1_y", "lower_inner1_z", 
    "lower_inner2_x", "lower_inner2_y", "lower_inner2_z",
    "upper_outer_x", "upper_outer_y", "upper_outer_z", 
    "upper_inner1_x", "upper_inner1_y", "upper_inner1_z", 
    "upper_inner2_x", "upper_inner2_y", "upper_inner2_z",
    "rc_x", "rc_y", "rc_z"
]
results_df = pd.DataFrame(results, columns=columns)

# Save the results as a CSV file (optional)
results_df.to_csv("suspension_simulation_results.csv", index=False)

# Display the first few rows
#print(results_df.head())


In [21]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=results_df["roll_angle"],
        y=results_df["rc_y"],
        mode='lines',
        marker=dict(color='red'),
        name='Roll Centre Location, Y',
        yaxis='y'  # Explicitly associate this trace with the primary y-axis
    )
)

# Add the second trace for "rc_z" on the secondary y-axis
fig.add_trace(
    go.Scatter(
        x=results_df["roll_angle"],
        y=results_df["rc_z"],
        mode='lines',
        marker=dict(color='orange'),
        name='Roll Centre Location, Z',
        yaxis='y2'  # Explicitly associate this trace with the secondary y-axis
    )
)

# Update layout to add a secondary y-axis
fig.update_layout(
    title='Roll Centre Location',
    xaxis=dict(title='Roll Angle (radians)'),
    yaxis=dict(
        title='RC Location, Y (m)',
        titlefont=dict(color='red'),
        tickfont=dict(color='red')
    ),
    yaxis2=dict(
        title='RC Location, Z (m)',
        titlefont=dict(color='orange'),
        tickfont=dict(color='orange'),
        anchor='x',
        overlaying='y',
        side='right'  # Place the secondary y-axis on the right side
    ),
    legend=dict(x=0.5, y=-0.2, orientation='h')  # Adjust legend position if needed
)

# Show the figure
fig.show()

In [22]:
def resolve_forces(tire_cop, lower_a_arm, spring_force, tire_force):
    """
    Resolve forces from the tire center of pressure (tire_cop) through the lower A-arm.
    
    Parameters:
        tire_cop (np.array): Coordinates of the tire center of pressure (1x3).
        lower_a_arm (np.array): Coordinates of the lower A-arm points (3x3):
                                [outboard, inboard_front, inboard_rear].
        spring_force (float): Force exerted by the spring (N).
        tire_force (float): Vertical force acting at the tire_cop (N).
    
    Returns:
        dict: Resolved forces at each point on the lower A-arm.
    """
    # Extract points
    outboard = lower_a_arm[0]
    inboard_front = lower_a_arm[1]
    inboard_rear = lower_a_arm[2]

    # Geometry vectors
    vec_out_to_front = inboard_front - outboard
    vec_out_to_rear = inboard_rear - outboard

    # Normalize vectors
    vec_out_to_front_unit = vec_out_to_front / np.linalg.norm(vec_out_to_front)
    vec_out_to_rear_unit = vec_out_to_rear / np.linalg.norm(vec_out_to_rear)

    # Assume vertical load (Fz) is fully carried by the outboard point
    # Force at the outboard is balanced by inboard forces and the spring
    force_outboard = np.array([0, 0, -tire_force])

    # Distribute outboard force between the inboard points based on geometry
    # Assume a linear distribution proportional to vector lengths
    front_contribution = np.dot(vec_out_to_front_unit, force_outboard)
    rear_contribution = np.dot(vec_out_to_rear_unit, force_outboard)

    # Scale contributions to sum to total tire force
    total_contribution = abs(front_contribution) + abs(rear_contribution)
    force_inboard_front = force_outboard * (abs(front_contribution) / total_contribution)
    force_inboard_rear = force_outboard * (abs(rear_contribution) / total_contribution)

    # Include spring force acting at the outboard
    force_inboard_front[2] += spring_force / 2
    force_inboard_rear[2] += spring_force / 2

    return {
        "force_outboard": force_outboard,
        "force_inboard_front": force_inboard_front,
        "force_inboard_rear": force_inboard_rear,
    }

# Resolve forces
forces = resolve_forces(tire_cop, lower_a_arm, spring_force, tire_force)
print("Resolved Forces:", forces)


Resolved Forces: {'force_outboard': array([    0,     0, -5000]), 'force_inboard_front': array([    0.        ,     0.        , -2035.77171589]), 'force_inboard_rear': array([    0.        ,     0.        , -1964.22828411])}
